In [6]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import sys
sys.path.insert(0, '../../../')

import numpy as np
import matplotlib.pyplot as plt
import pickle

from src.difsched.config import getSubAgentConfig, visualizeSubAgentConfig, getEnvConfig, visualizeEnvConfig
from src.difsched.env import createEnv
from src.difsched.agents.mdp import MdpKernel
from src.difsched.env import PolicySimulator
from src.difsched.agents.Others import RandomPolicy

In [7]:
def genDatasetByPolicies(
        policies, userMap, num_windows,
        env,
        N_episodes = 10,
        obvMode = "predicted",
        mode="train",
        type="data"
    ):
    len_episode = int(num_windows/N_episodes)
    simResult = {'uRecord': [], 'actionsRecord': [], 'rewardRecord': [], 'uNextRecord': []}
    for i in range(N_episodes):
        policySimulator = PolicySimulator(env)
        policySimulator.loadPolicies(policies, userMap)
        policySimulator.setupModes(obvMode=obvMode, mode=mode, type=type)
        simResult_ep = policySimulator.runSimulation(num_windows=len_episode, N_episodes=N_episodes)
        simResult['actionsRecord'] += simResult_ep['actionsRecord']
        simResult['rewardRecord'] += simResult_ep['rewardRecord']
        simResult['uNextRecord'] += simResult_ep['uNextRecord']
        simResult['uRecord'] += simResult_ep['uRecord']
    return simResult

def load_mdp_policy(configIdx):
    with open(f'../../../data/results/MdpPolicy/mdp_config{configIdx}.pkl', 'rb') as f:
        mdpKernelParams = pickle.load(f)
    mdpPolicy = MdpKernel()
    mdpPolicy.load_policy(mdpKernelParams, policyMode="deterministic", randomR=False)
    return mdpPolicy

In [9]:
num_windows = 10000
N_episodes = 10
len_episode = int(num_windows/N_episodes)
obvMode = "predicted" # "perfect" or "predicted"
savePath = f'../../../data/processed/offline_dataset'

for envConfigIdx in [0,1]:
    envParams = getEnvConfig(envConfigIdx)
    visualizeEnvConfig(envParams)
    trafficDataParentPath =  f'../../../data/raw/traffic'

    env = createEnv(envParams, trafficDataParentPath)

    usermap = envParams['user_map']
    poliocy_set = envParams['sub_agents_idx']
    simResultTotal = {'uRecord': [], 'actionsRecord': [], 'rewardRecord': [], 'uNextRecord': []}
    for sub_agent_idxs in poliocy_set:
        policies = []
        print(f"sub_agent_idxs: {sub_agent_idxs}")
        for sub_agent_idx in sub_agent_idxs:
            subAgentParams = getSubAgentConfig(sub_agent_idx)
            visualizeSubAgentConfig(subAgentParams) 
            policies.append(load_mdp_policy(sub_agent_idx))

        simResult = genDatasetByPolicies(
            policies, usermap, num_windows, 
            env, 
            N_episodes = N_episodes, obvMode = obvMode, mode="train", type="data")
        simResultTotal['uRecord'] += simResult['uRecord']
        simResultTotal['actionsRecord'] += simResult['actionsRecord']
        simResultTotal['rewardRecord'] += simResult['rewardRecord']
        simResultTotal['uNextRecord'] += simResult['uNextRecord']
        
    print(f"Avg. packet loss rate: {np.mean(simResultTotal['rewardRecord'])}")
    print(f"length of dataset: {len(simResultTotal['uRecord'])}")
    with open(f'{savePath}/subOptimalAgent_encConfig{envConfigIdx}_{poliocy_set}.pkl', 'wb') as f:
        pickle.dump(simResultTotal, f)
    print(f"saved to {savePath}/subOptimalAgent_encConfig{envConfigIdx}_{poliocy_set}.pkl")

Environment Configuration
Number of Users:        8
Window Length:          200
Dataflow:               thumb_fr
Sigmoid K List:         [0.1, 0.2, 0.3, 0.4, 0.5]
Sigmoid S List:         [10.0, 10.0, 10.0, 10.0, 10.0]
Resource Bar:           5
Bandwidth:              100
Sub Agents:             [[0, 0]]
User Map:               [[0, 1, 2, 3], [4, 5, 6, 7]]
sub_agent_idxs: [0, 0]
Environment Configuration
Number of Users:        4
Window Length:          200
Dataflow:               thumb_fr
N_aggregation:          4
N_r:                    5
Resource Bar:           5
Bandwidth:              60
Sigmoid K List:         [0.3]
Sigmoid S List:         [10.0]
Random Seed:            999
Environment Configuration
Number of Users:        4
Window Length:          200
Dataflow:               thumb_fr
N_aggregation:          4
N_r:                    5
Resource Bar:           5
Bandwidth:              60
Sigmoid K List:         [0.3]
Sigmoid S List:         [10.0]
Random Seed:            999


Avg. packet loss rate: 0.33388894431115085
length of dataset: 10000
saved to ../../../data/processed/offline_dataset/subOptimalAgent_encConfig0_[[0, 0]].pkl
Environment Configuration
Number of Users:        20
Window Length:          200
Dataflow:               thumb_fr
Sigmoid K List:         [0.1, 0.2, 0.3, 0.4, 0.5]
Sigmoid S List:         [10.0, 10.0, 10.0, 10.0, 10.0]
Resource Bar:           5
Bandwidth:              200
Sub Agents:             [[0, 0, 0, 0, 0]]
User Map:               [[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11], [12, 13, 14, 15], [16, 17, 18, 19]]
sub_agent_idxs: [0, 0, 0, 0, 0]
Environment Configuration
Number of Users:        4
Window Length:          200
Dataflow:               thumb_fr
N_aggregation:          4
N_r:                    5
Resource Bar:           5
Bandwidth:              60
Sigmoid K List:         [0.3]
Sigmoid S List:         [10.0]
Random Seed:            999
Environment Configuration
Number of Users:        4
Window Length:          200
Data

Avg. packet loss rate: 0.45128035676105227
length of dataset: 10000
saved to ../../../data/processed/offline_dataset/subOptimalAgent_encConfig1_[[0, 0, 0, 0, 0]].pkl


In [16]:
envConfigIdx = 0
envParams = getEnvConfig(envConfigIdx)
visualizeEnvConfig(envParams)
with open(f'../../../data/processed/offline_dataset/subOptimalAgent_encConfig{envConfigIdx}_{envParams["sub_agents_idx"]}.pkl', 'rb') as f:
    dataset = pickle.load(f)
print(f"Size of dataset: {len(dataset['uRecord'])}")
print(np.mean(dataset['rewardRecord']))

Environment Configuration
Number of Users:        8
Window Length:          200
Dataflow:               thumb_fr
Sigmoid K List:         [0.1, 0.2, 0.3, 0.4, 0.5]
Sigmoid S List:         [10.0, 10.0, 10.0, 10.0, 10.0]
Resource Bar:           5
Bandwidth:              100
Sub Agents:             [[0, 0]]
User Map:               [[0, 1, 2, 3], [4, 5, 6, 7]]
Size of dataset: 10000
0.33388894431115085
